# Imports

In [1]:
import csv
import h5py
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pylab
import cv2

from scipy.stats import moment 
from scipy.ndimage import shift

from imblearn.over_sampling import RandomOverSampler

from keras import regularizers, optimizers
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Activation, Input, Embedding, concatenate
from keras.models import load_model, Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.models import Model

import xgboost as xgb

from skimage import segmentation, morphology, measure
from skimage.transform import resize
from skimage.morphology import watershed

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

Using TensorFlow backend.


# Set paths and model hyper-parameters

In [17]:
class CONFIG():
    def __init__(self, path, data_location='data/', train_images_location='data/train_images/',
                 test_images_location='/data/test_images/', image_size=56, n_classes=121, batch_size=64,
                 n_epochs=10):
        self.path = path
        self.data_location = self.path + data_location
        self.train_images_location = self.path + train_images_location
        self.test_images_location = self.path + test_images_location 
        self.image_size = image_size
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.n_epochs = n_epochs

In [18]:
cfg = CONFIG(path='/Users/guillaumecorda/Desktop/UvA/Applied Machine Learning/Kaggle/aml_kaggle/')

In [19]:
train_dir = '/Users/guillaumecorda/Desktop/UvA/Applied Machine Learning/Kaggle/data/train_images/'
validation_dir = '/Users/guillaumecorda/Desktop/UvA/Applied Machine Learning/Kaggle/data/val_images/'

# Load and process image data and labels

## Load data

In [20]:
# Load train images filenames with class labels
filenames = [i for i in os.listdir(cfg.path+'data/train_images') if i.endswith('.jpg')]
with open(cfg.data_location + 'train_onelabel.csv', mode='r') as infile:
    reader = csv.reader(infile)
    file_to_class = {rows[0]:rows[1] for rows in reader}

# Calculate new class counts, converging towards max (1580)
with open(cfg.data_location + 'train_onelabel.csv', mode='r') as infile:
    reader = csv.reader(infile)
    class_counts = {}
    for row in reader:
        if(row[1] != 'class'):
            class_counts[int(row[1])] = class_counts.get(int(row[1]), 0) + 1
    max_nr = max(class_counts.values())
    for key, value in class_counts.items():
        class_counts[key] = int(class_counts[key] + (max_nr - class_counts[key])/6)

In [21]:
X = np.empty([len(filenames),cfg.image_size, cfg.image_size, 1])
Y_tmp = np.empty([len(filenames)])
Y = np.empty([sum(class_counts.values()),cfg.n_classes])
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y.shape))

Shapes:
X:(24204, 56, 56, 1)
Y:(51985, 121)


## Helper functions

In [7]:
def get_padding(i):
    """
    Helper function for getting right padding sizes
    input:
        - i: positive integer gotten from substracting height and width of an image
    output:
        - Tuple representing the correct padding
    """
    if i%2 == 0:
        return (int(i/2),int(i/2))
    else:
        return (int(i/2-.5), int(i/2+.5))

def pad_image(img):
    """
    Add padding to image to make it square
    input:
        - img: numpy array (2D) representing image
    output:
        - padded array of shape (N,N)
    """
    H, W = img.shape
    if H == W:
        return img
    elif H > W:
        return np.pad(img, ((0,0), get_padding(H-W)), 'constant')
    else:
        return np.pad(img, (get_padding(W-H), (0,0)), 'constant')

def resize_image(img):
    """
    Resize image to new square shape
    input:
        - img: numpy array (2D) representing image
        - size: final shape of image in pixels (integer)
    """
    return resize(img, (cfg.image_size, cfg.image_size), mode='reflect')


def find_highest_dimensions(path):
    
    path = cfg.path+path
    images_names = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    max_rows = 0
    max_cols = 0
    
    for image in images_names:
        try:
            im = np.asarray(PIL.Image.open(path+image))
            if im.shape[0] > max_rows:
                max_rows = im.shape[0]
            if im.shape[1] > max_cols:
                max_cols = im.shape[1]
        except:
            print('Image {} failed.\n'.format(image))
    max_shape = (max_rows, max_cols)
    
    return max_shape

## Preprocess images

For image in filenames:
- load file
- from [0,255] to [0.0 to 1.0]
- square and resize image
- either:
    - add image once (X & Y)
- or:
    - rotate [0,90,180,270]
    - add 4 images to X
    - add 4 labels to Y


In [22]:
total = len(filenames)
for i in range(len(filenames)):
    # read and transform image to usable format
    img = mpimg.imread(cfg.train_images_location + filenames[i])
    img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
    img = resize_image(pad_image(img))
    # create a grayscale channel 
    img = img.reshape(cfg.image_size, cfg.image_size, 1)
    
    X[i] = img
    Y_tmp[i] = int(file_to_class[filenames[i]])
    
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y.shape))

Shapes:
X:(24204, 56, 56, 1)
Y:(51985, 121)


### Compute New Features

In [23]:
def compute_new_features(filenames):
    N = len(filenames)
    output = np.zeros((N,9))
    for i, image in enumerate(filenames):
        image = mpimg.imread(cfg.train_images_location + filenames[i])
        h = image.shape[0]
        w = image.shape[1]
        h_ = h/w
        w_ = w/h
        h_2 = (h/w)**2
        w_2 = (w/h)**2
        m = np.mean(image)
        moment_2 = moment(image.flatten(), moment=2)
        moment_3 = moment(image.flatten(), moment=3)
        output[i] = np.array([h, w, h_, w_, h_2, w_2, m, moment_2, moment_3])
    return output

In [ ]:
new_features = compute_new_features(filenames)

### Plot Height/Widths Amongst Classes 

In [ ]:
plt.scatter(new_features[:,0], new_features[:,1], c=Y_tmp.tolist(), cmap=pylab.cm.cool)

### Find optimal image size

In [ ]:
plt.hist(new_features[:,0])

In [ ]:
mean_height = np.median(new_features[:,0])
mean_height

In [ ]:
plt.hist(new_features[:,1])

In [ ]:
mean_width = np.median(new_features[:,1])
mean_width

### Example image

In [ ]:
f = plt.figure(figsize=(16,3))
sub1 = plt.subplot(1,4,1)
plt.imshow(X[250][:,:,0], cmap='binary')

In [ ]:
val = shift(X[250][:,:,0], -10)
plt.imshow(val, cmap='binary')

### Rotations and shifts

https://github.com/aleju/imgaug

In [24]:
X_augm = np.concatenate((X, X, X, X, X, X, X, X))

Y_augm = np.concatenate((Y_tmp, Y_tmp, Y_tmp, Y_tmp, Y_tmp, Y_tmp, Y_tmp, Y_tmp))

In [25]:
# rotate images by one of 0/90/180/270 degrees
for i in range(X.shape[0], 2*X.shape[0]):
    X_augm[i] = np.rot90(X_augm[i],(1+(i%4)))
    
#flip images
for i in range(2*X.shape[0], 3*X.shape[0]):
    X_augm[i] = np.fliplr(X_augm[i])

#flip and rotation images
for i in range(3*X.shape[0], 4*X.shape[0]):
    X_augm[i] = np.fliplr(X_augm[i])
    X_augm[i] = np.rot90(X_augm[i],(1+(i%4)))

#flip images
for i in range(4*X.shape[0], 5*X.shape[0]):
    X_augm[i] = np.flipud(X_augm[i])

#flip and rotation images
for i in range(5*X.shape[0], 6*X.shape[0]):
    X_augm[i] = np.flipud(X_augm[i])
    X_augm[i] = np.rot90(X_augm[i],(1+(i%4)))

#shift
for i in range(6*X.shape[0], 7*X.shape[0]):
    #shift = np.random.uniform(low=-cfg.image_size//2, high=cfg.image_size//2)
    val = shift(X_augm[i][:,:,0], 10)
    X_augm[i] = val.reshape(cfg.image_size, cfg.image_size, 1)

In [ ]:
#Check transformations
for i in range(7):
    f = plt.figure(figsize=(16,3))
    sub1 = plt.subplot(1,4,1)
    plt.imshow(X_augm[i*total][:,:,0], cmap='binary')
    plt.imshow(X_augm[i*total][:,:,0], cmap='binary')

# Class imbalance
Since there is a strong class imbalance (lowest 7, highest 1580), something has to be done to counter this. Oversampling minority classes to be as big as the majority classes is the option used below. <br>
(Please do note that the validation split accuracy can not be seen as a surrogate for the test set.)

## RandomOverSampler

In [ ]:
X = X.reshape(total,cfg.image_size*cfg.image_size)
print(X.shape)

sm = RandomOverSampler(ratio=class_counts)
X, Y_tmp = sm.fit_sample(X, Y_tmp)
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y_tmp.shape))

X = X.reshape(len(X),cfg.image_size,cfg.image_size,1)
for i in range(len(Y_tmp)):
    Y[i][int(Y_tmp[i])] = 1.0
del(Y_tmp)
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y.shape))

In [ ]:
for i in range(total, X.shape[0]):
    # rotate RandomOverSampler images by one of 0/90/180/270 degrees
    X[i] = np.rot90(X[i],(1+(i%4)))

Uncomment the lines below appropriately if a validation split is to be used.

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
# only uncomment if train/validation variables are used
# del(X)
# del(Y)

## SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X_augm = X_augm.reshape((X_augm.shape[0], X_augm.shape[1]*X_augm.shape[2]))

In [ ]:
X_augm.shape

In [ ]:
#ratio : the desired ratio of the number of samples in the majority class 
#        over the number of samples in the minority class after resampling.
X_resampled, Y_resampled = SMOTE(random_state=42).fit_resample(X_augm, Y_augm)

In [ ]:
X_resampled = X_resampled.reshape((X_resampled.shape[0], cfg.image_size, cfg.image_size, 1))

In [ ]:
X_resampled.shape

In [ ]:
Y_resampled.shape

In [ ]:
Y_resampled[0]

In [ ]:
Y_resampled[-1]

# Format target

In [26]:
def format_targets(y_true):
    N = len(y_true)
    output = np.zeros(shape=(N, 121))
    for i in range(N):
        j=0
        while j != y_true[i]:
            j+=1
        output[i][j]=1
    return output

In [27]:
#to do if RandomOverSampling not used
#Y_resampled = format_targets(Y_resampled)
Y_augm = format_targets(Y_augm)

In [ ]:
Y_tmp = format_targets(Y_tmp)

# Classifiers :

## CNN no reg

In [28]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=X_augm[0].shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(8, kernel_size=(3, 3), padding='same', activation='relu'))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(cfg.n_classes, activation='softmax'))

#optimizer = optimizers.Adam(lr=0.0001)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 56, 56, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 8)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1568)              0         
__________

In [29]:
history = model.fit(
    X_augm, 
    Y_augm,
    epochs=5, 
    batch_size=cfg.batch_size,
    verbose=1,
    class_weight=class_counts)

Epoch 1/5
  2880/193632 [..............................] - ETA: 25:35 - loss: 2690.6686 - acc: 0.0979

KeyboardInterrupt: 

## CNN

### Train

In [15]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), padding='same', input_shape=X_augm[0].shape))
                 #kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3, 3), padding='same'))#, kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3, 3), padding='same'))#, kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))#, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))#, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(cfg.n_classes, activation='softmax'))

optimizer = optimizers.Adam(lr=0.0001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 16)        160       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 16)        0         
__________

In [16]:
history = model.fit(
    X_augm, 
    Y_augm,
    epochs=5, 
    batch_size=cfg.batch_size,
    verbose=1,
    class_weight=class_counts)

Epoch 1/5
193632/193632 [==============================] - 720s 4ms/step - loss: 2198.4603 - acc: 0.1814
Epoch 2/5
193632/193632 [==============================] - 663s 3ms/step - loss: 1673.4949 - acc: 0.2761
Epoch 3/5
193632/193632 [==============================] - 692s 4ms/step - loss: 1483.3934 - acc: 0.3187
Epoch 4/5
193632/193632 [==============================] - 697s 4ms/step - loss: 1375.0587 - acc: 0.3483
Epoch 5/5
129280/193632 [===================>..........] - ETA: 4:31 - loss: 1309.6124 - acc: 0.3679

KeyboardInterrupt: 

**Try to change optimizer to adam**

In [ ]:
best_model = load_model(cfg.path+'/output_guillaume/models/model_3_BEST.h5')

In [ ]:
history = model.fit(
    X_augm, 
    Y_augm,
    epochs=3, 
    batch_size=cfg.batch_size,
    verbose=1,
    class_weight=class_counts)

In [ ]:
model.save(cfg.path+'/output_guillaume/models/model_5.h5')

### Predict classes

In [ ]:
filenames = [i for i in os.listdir(cfg.path+'/data/test_images') if i.endswith('.jpg')]

labels = pd.DataFrame(filenames, columns=['image'])
labels['class'] = -1

In [ ]:
total = len(filenames)
for i in range(total):
    # read and transform image to usable format
    img = mpimg.imread(cfg.test_images_location + filenames[i])
    img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
    img = resize_image(pad_image(img))
    # uncomment next line if ConvLayer in Model 
    img = img.reshape(1,cfg.image_size, cfg.image_size,1)
    # uncomment next line if no ConvLayer in Model
    # img = img.flatten().reshape([-1,4096])
    
    labels.loc[labels['image'] == filenames[i], 'class'] = model.predict_classes(img, verbose=0)[0]

labels.sort_values(by='class')
labels['class'] = labels['class'].astype(int)
labels.sample(n=5)

In [ ]:
labels

In [ ]:
labels.to_csv(cfg.path+'output_guillaume/predictions/model_3.csv', index=False)

## Multiple Input Network

https://keras.io/getting-started/functional-api-guide/

### Load CNN

In [ ]:
main_input = Input(shape=X_augm[0].shape, name='main_input')

#cnn_model = load_model(cfg.path+'/output_guillaume/models/model_5.h5')

for layer in model.layers:
    print(layer.name)

In [ ]:
cnn_layer_output = Model(inputs=model.input, outputs=model.get_layer('dropout_24').output)
cnn_layer_output = cnn_layer_output.predict(X_augm)

main_model = cnn_layer_output

### Load FC for new_features

In [ ]:
fc_model = Sequential()

fc_model.add(Dense(512, activation='relu', input_shape=new_features[0].shape))#, kernel_regularizer=regularizers.l2(0.01)))
fc_model.add(Dropout(0.5))
fc_model.add(Dense(512, activation='relu'))#, kernel_regularizer=regularizers.l2(0.01)))
fc_model.add(Dropout(0.5))
fc_model.add(Dense(512, activation='relu'))#, kernel_regularizer=regularizers.l2(0.01)))
fc_model.add(Dropout(0.5))

fc_model.add(Dense(cfg.n_classes, activation='softmax'))

optimizer = optimizers.Adam(lr=0.0001)

fc_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

fc_model.summary()

In [ ]:
history = fc_model.fit(
    new_features, 
    Y_tmp,
    epochs=4, 
    batch_size=cfg.batch_size,
    verbose=1,
    class_weight=class_counts)

In [ ]:
fc_model.save(cfg.path+'/output_guillaume/models/model_fc.h5')

In [ ]:
aux_input = Input(shape=new_features[0].shape, name='aux_input')

#fc_model = load_model(cfg.path+'/output_guillaume/models/model_fc.h5')

for layer in fc_model.layers:
    print(layer.name)

In [ ]:
fc_layer_output = Model(inputs=fc_model.input, outputs=fc_model.get_layer('dropout_37').output)
fc_layer_output = fc_layer_output.predict(new_features)

aux_model = fc_layer_output

In [ ]:
merged = concatenate([main_model, aux_model])

output = Dense(512, activation='relu')(merged)
output = Dense(cfg.n_classes, activation='softmax')(output)

In [ ]:
model = Model(inputs=[main_input, aux_input], outputs=[output])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([X_augm, new_features], [Y_augm, Y_tmp], epochs=5, batch_size=cfg.batch_size)

## Transfer Learning

In [ ]:
cnn_model = load_model(cfg.path+'/output_guillaume/models/model_3_BEST.h5')

In [ ]:
def get_feature_layer(data):
    
    total_layers = len(model.layers)
    
    fl_index = total_layers-3
    
    feature_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer(index=fl_index).output)
    
    feature_layer_output = feature_layer_model.predict(data)
    
    return feature_layer_output

In [ ]:
def train_xgb_cnn(x_train_cnn, y_train, cnn=None, num_round=100):
    param = {'eta':0.1,
             'max_depth':6,
             'objective':'multi:softmax',
             'n_estimators':175,
             'silent':1,
             'num_class':cfg.n_classes}
    feat_out = get_feature_layer(cnn_model, x_train_cnn)
    dtrain = xgboost.DMatrix(feat_out,
                             label=y_train.idxmax(axis=1).values)
    xgb_feature_layer = xgboost.train(param, dtrain, num_round)
    return xgb_feature_layer

In [ ]:
get_feature_layer(X)

In [ ]:
base_model = load_model(cfg.path+'/output_guillaume/models/model_3.h5')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

## Transform and train on the fly

In [ ]:
model = load_model(cfg.path+'/output_guillaume/models/model_4.h5')
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=90,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(cfg.image_size, cfg.image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(cfg.image_size, cfg.image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=cfg.n_epochs,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

In [ ]:
model.save(cfg.path+'/output_guillaume/models/model_4.h5')

# Hidden Layers Output Analysis

In [ ]:
# Load model
model_3 = load_model(cfg.path+'/output_guillaume/models/model_3_BEST.h5')
 
# Show a summary of the model. Check the number of trainable parameters
model_3.summary()

In [ ]:
#model is the model you wanna extract the hidden layers output 
layer_names = ['conv2d_5', 'conv2d_6', 'conv2d_7']
intermediate_layer_model = model_3.get_layer(layer_names[0]).output
intermediate_output = intermediate_layer_model.predict(data)

In [ ]:
keras.

In [ ]:
intermediate_layer_model.output

In [ ]:
for layer in layer_names:
    intermediate_layer_model = model_3.get_layer(layer).output

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=90,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 1000
val_batchsize = 100
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(cfg.image_size, cfg.image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(cfg.image_size, cfg.image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)